### imports

In [1]:
import pandas as pd
import spacy
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt

/Applications/anaconda3/envs/mina/lib/python3.13/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


In [3]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters

# Explicitly load the correct Punkt model from the 'punkt' package
nltk.download('punkt')

# Manually instantiate the tokenizer
tokenizer = PunktSentenceTokenizer()

# Use it directly if needed
text = "This is a test sentence. Let's see if it works!"
sentences = tokenizer.tokenize(text)
print("Sentences:", sentences)

# Then use word_tokenize as usual
print("Words:", word_tokenize(text))

Sentences: ['This is a test sentence.', "Let's see if it works!"]


[nltk_data] Downloading package punkt to /Users/Devyani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/Devyani/nltk_data'
**********************************************************************


In [2]:
from textstat import flesch_reading_ease
from collections import Counter
import numpy as np
from nltk.util import ngrams
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

In [10]:
from nltk.tokenize import word_tokenize
from textstat import flesch_reading_ease

In [3]:
import json

### taking a closer look at cleaned data

In [4]:
df = pd.read_csv('coauthor_combined_cleaned.csv')

In [5]:
df.head()

,eventName,eventSource,eventTimestamp,textDelta,cursorRange,currentDoc,currentCursor,currentSuggestions,currentSuggestionIndex,currentHoverIndex,currentN,currentMaxToken,currentTemperature,currentTopP,currentPresencePenalty,currentFrequencyPenalty,eventNum,session_id
0,system-initialize,api,2021-08-16 07:00:41.033,NaN,NaN,"A woman has been dating guy after guy, but it ...",244,[],0,NaN,5,30,0.3,1,0,0.0,0,e0435f4cf6fc435c872ffc5b66b66b0c
1,text-insert,user,2021-08-16 07:00:46.487,"{'ops': [{'retain': 244}, {'insert': ' '}]}",NaN,NaN,245,[],0,NaN,5,30,0.3,1,0,0.0,1,e0435f4cf6fc435c872ffc5b66b66b0c
2,text-insert,user,2021-08-16 07:00:46.731,"{'ops': [{'retain': 245}, {'insert': ' '}]}",NaN,NaN,246,[],0,NaN,5,30,0.3,1,0,0.0,2,e0435f4cf6fc435c872ffc5b66b66b0c
3,text-insert,user,2021-08-16 07:00:46.897,"{'ops': [{'retain': 246}, {'insert': ' '}]}",NaN,NaN,247,[],0,NaN,5,30,0.3,1,0,0.0,3,e0435f4cf6fc435c872ffc5b66b66b0c
4,text-delete,user,2021-08-16 07:00:47.247,"{'ops': [{'retain': 246}, {'delete': 1}]}",NaN,NaN,246,[],0,NaN,5,30,0.3,1,0,0.0,4,e0435f4cf6fc435c872ffc5b66b66b0c


In [6]:
df.describe()

,currentCursor,currentSuggestionIndex,currentHoverIndex,currentN,currentMaxToken,currentTemperature,currentTopP,currentPresencePenalty,currentFrequencyPenalty,eventNum
count,2.701458e+06,2.701458e+06,2.299216e+06,2701458.0,2701458.0,2.701458e+06,2701458.0,2701458.0,2.701458e+06,2.701458e+06
mean,1.413145e+03,7.657195e-01,1.691181e+00,5.0,30.0,5.214587e-01,1.0,0.0,4.902231e-01,1.065809e+03
std,7.792813e+02,1.238773e+00,1.432100e+00,0.0,0.0,2.810858e-01,0.0,0.0,3.878369e-01,7.630140e+02
min,0.000000e+00,0.000000e+00,0.000000e+00,5.0,30.0,2.000000e-01,1.0,0.0,0.000000e+00,0.000000e+00
25%,8.050000e+02,0.000000e+00,0.000000e+00,5.0,30.0,3.000000e-01,1.0,0.0,0.000000e+00,4.660000e+02
50%,1.314000e+03,0.000000e+00,2.000000e+00,5.0,30.0,3.000000e-01,1.0,0.0,5.000000e-01,9.440000e+02
75%,1.880000e+03,1.000000e+00,3.000000e+00,5.0,30.0,7.500000e-01,1.0,0.0,1.000000e+00,1.512000e+03
max,5.679000e+03,4.000000e+00,4.000000e+00,5.0,30.0,9.000000e-01,1.0,0.0,1.000000e+00,6.621000e+03


In [7]:
print(df.columns.tolist())

['eventName', 'eventSource', 'eventTimestamp', 'textDelta', 'cursorRange', 'currentDoc', 'currentCursor', 'currentSuggestions', 'currentSuggestionIndex', 'currentHoverIndex', 'currentN', 'currentMaxToken', 'currentTemperature', 'currentTopP', 'currentPresencePenalty', 'currentFrequencyPenalty', 'eventNum', 'session_id']


### filter 

In [12]:
df = df[
    (df['eventName'] == 'text-insert') &
    (df['eventSource'].isin(['user', 'model'])) &
    (df['textDelta'].notna()) &
    (df['textDelta'].str.strip() != "")
].copy()

In [9]:
print(df.columns.tolist())


['eventName', 'eventSource', 'eventTimestamp', 'textDelta', 'cursorRange', 'currentDoc', 'currentCursor', 'currentSuggestions', 'currentSuggestionIndex', 'currentHoverIndex', 'currentN', 'currentMaxToken', 'currentTemperature', 'currentTopP', 'currentPresencePenalty', 'currentFrequencyPenalty', 'eventNum', 'session_id']


### analysis

In [13]:
def compute_stylometric_features(text):
    tokens = word_tokenize(text)
    words = [word for word in tokens if word.isalpha()]
    stop_words = set(stopwords.words('english'))

    return {
        'readability': flesch_reading_ease(text),
        'avg_word_length': np.mean([len(w) for w in words]) if words else 0,
        'lexical_diversity': len(set(words)) / len(words) if words else 0,
        'punctuation_freq': sum(1 for c in text if c in string.punctuation) / len(text) if text else 0,
        'stopword_ratio': sum(1 for w in words if w.lower() in stop_words) / len(words) if words else 0
    }

In [15]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Devyani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
import nltk
nltk.data.path.append('/Users/Devyani/nltk_data')

In [18]:
features_list = []
for _, row in df.iterrows():
    text = str(row['textDelta']) 
    source = row['eventSource']
    features = compute_stylometric_features(text)
    features['source'] = source
    features_list.append(features)

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/Users/Devyani/nltk_data'
    - '/Applications/anaconda3/envs/mina/nltk_data'
    - '/Applications/anaconda3/envs/mina/share/nltk_data'
    - '/Applications/anaconda3/envs/mina/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/Users/Devyani/nltk_data'
**********************************************************************
